* در مواقعی که جداول MySQL را در حالت MyISAM قرار دهید، به خاطر عدم پشتیبانی از MVCC و پیچیدگی‌های آن، سرعت بالاتری نسبت در هنگام READ نسبت به Postgres دارد


### Create Or Update : 
```
INSERT INTO users (id, name, email)
VALUES (123, 'John Doe', 'john.doe@example.com')
ON DUPLICATE KEY UPDATE name = 'John Doe', email = 'john.doe@example.com';
```


### Drop duplicated : 
```
SELECT DISTINCT city
FROM Employee;
```

### Operator : AND, OR, In, Not In, BETWEEN, Null
```
SELECT COUNT(*) FROM `Invoice`
WHERE NOT (InvoiceDate > '2013-01-01' and InvoiceDate < '2013-12-30');

SELECT count(*) FROM `payment`
WHERE payment_date BETWEEN '2005-05-26' and '2005-05-27';

SELECT * FROM `address`
WHERE district in ("Texas","QLD");

```

### Like, REGEXP Operator:

 - % : any    character
 - _ : single character
  - 
```
SELECT * FROM `actor` WHERE last_name LIKE 'H____S';
SELECT * FROM `actor` WHERE last_name REGEXP 'pattern'
```

### OrderBy
```
SELECT * FROM `rental` ORDER BY inventory_id + customer_id ASC;
```

### Limit
```
SELECT * FROM `actor` LIMIT 4(from),5(count);
```

### Composite Primary key Join
```
SELECT * FROM rental r
JOIN payment p 
	ON p.payment_id = r.rental_id
    AND p.customer_id = r.inventory_id
```

### Implicit join, and the USING
```
SELECT c.customer_id, sum(p.amount) FROM payment p, customer c
WHERE p.customer_id = c.customer_id
GROUP BY c.customer_id;


SELECT p.payment_id, c.customer_id, p.amount 
FROM customer c
JOIN payment p 
USING (customer_id);


```

### OuterJoin: Left, Right ------- InnerJoin: join


